In [11]:
import spark_bindings

from pyspark.sql import SparkSession
from pyspark import SparkConf

from pyspark.sql.types import *
import json

In [2]:
sconf = SparkConf()

sconf.set('spark.submit.deployMode','client')
sconf.set('spark.master','yarn')

app_name="I am Spark"
sconf.set('spark.app.name',app_name)

spark = ( SparkSession
    .builder
    .config(conf=sconf)
    .enableHiveSupport()
    .getOrCreate()
)

In [4]:
from avro.schema import parse

# copy paste from schema reg
my_avro_schema_string = """{"name":"Vaerutsatt_veg","type":"record","fields":[{"name":"type","type":["null","string"],"default":null},{"name":"vegobjekt","type":["null",{"type":"record","name":"vegobjekt","fields":[{"name":"id","type":["null","int"],"default":null},{"name":"href","type":["null","string"],"default":null},{"name":"metadata","type":["null",{"type":"record","name":"vegobjekt","namespace":"metadata","fields":[{"name":"type","type":["null",{"type":"record","name":"vegobjekt","namespace":"type.metadata","fields":[{"name":"id","type":["null","int"],"default":null},{"name":"navn","type":["null","string"],"default":null}]}]},{"name":"versjon","type":["null","int"],"default":null},{"name":"startdato","type":["null","string"],"default":null},{"name":"sist_modifisert","type":["null","string"],"default":null}]}]},{"name":"geometri","type":["null",{"type":"record","name":"vegobjekt","namespace":"geometri","fields":[{"name":"wkt","type":["null","string"],"default":null},{"name":"srid","type":["null","int"],"default":null},{"name":"egengeometri","type":["null","boolean"],"default":null}]}]},{"name":"lokasjon","type":["null",{"type":"record","name":"vegobjekt","namespace":"lokasjon","fields":[{"name":"kommuner","type":["null",{"type":"array","items":"int"}]},{"name":"fylker","type":["null",{"type":"array","items":"int"}]},{"name":"regioner","type":["null",{"type":"array","items":"int"}]},{"name":"vegavdelinger","type":["null",{"type":"array","items":"int"}]},{"name":"kontraktsomraader","type":["null",{"type":"array","items":["null",{"type":"record","name":"vegobjekt","namespace":"kontraktsomraader.lokasjon","fields":[{"name":"nummer","type":["null","int"],"default":null},{"name":"navn","type":["null","string"],"default":null}]}]}]},{"name":"vegreferanser","type":["null",{"type":"array","items":["null",{"type":"record","name":"vegobjekt","namespace":"vegreferanser.lokasjon","fields":[{"name":"fylke","type":["null","int"],"default":null},{"name":"kommune","type":["null","int"],"default":null},{"name":"kategori","type":["null","string"],"default":null},{"name":"status","type":["null","string"],"default":null},{"name":"nummer","type":["null","int"],"default":null},{"name":"hp","type":["null","int"],"default":null},{"name":"fra_meter","type":["null","int"],"default":null},{"name":"til_meter","type":["null","int"],"default":null},{"name":"kortform","type":["null","string"],"default":null}]}]}]},{"name":"stedfestinger","type":["null",{"type":"array","items":["null",{"type":"record","name":"vegobjekt","namespace":"stedfestinger.lokasjon","fields":[{"name":"veglenkeid","type":["null","int"],"default":null},{"name":"fra_posisjon","type":["null","double"],"default":null},{"name":"til_posisjon","type":["null","double"],"default":null},{"name":"kortform","type":["null","string"],"default":null},{"name":"retning","type":["null","string"],"default":null}]}]}]},{"name":"geometri","type":["null",{"type":"record","name":"vegobjekt","namespace":"geometri.lokasjon","fields":[{"name":"wkt","type":["null","string"],"default":null},{"name":"srid","type":["null","int"],"default":null}]}]},{"name":"strekningslengde","type":["null","int"],"default":null}]}]},{"name":"vegsegmenter","type":["null",{"type":"array","items":["null",{"type":"record","name":"vegobjekt","namespace":"vegsegmenter","fields":[{"name":"stedfesting","type":["null",{"type":"record","name":"vegobjekt","namespace":"stedfesting.vegsegmenter","fields":[{"name":"veglenkeid","type":["null","int"],"default":null},{"name":"fra_posisjon","type":["null","double"],"default":null},{"name":"til_posisjon","type":["null","double"],"default":null},{"name":"kortform","type":["null","string"],"default":null},{"name":"retning","type":["null","string"],"default":null}]}]},{"name":"geometri","type":["null",{"type":"record","name":"vegobjekt","namespace":"geometri.vegsegmenter","fields":[{"name":"wkt","type":["null","string"],"default":null},{"name":"srid","type":["null","int"],"default":null}]}]},{"name":"kommune","type":["null","int"],"default":null},{"name":"fylke","type":["null","int"],"default":null},{"name":"region","type":["null","int"],"default":null},{"name":"vegavdeling","type":["null","int"],"default":null},{"name":"vegreferanse","type":["null",{"type":"record","name":"vegobjekt","namespace":"vegreferanse.vegsegmenter","fields":[{"name":"fylke","type":["null","int"],"default":null},{"name":"kommune","type":["null","int"],"default":null},{"name":"kategori","type":["null","string"],"default":null},{"name":"status","type":["null","string"],"default":null},{"name":"nummer","type":["null","int"],"default":null},{"name":"hp","type":["null","int"],"default":null},{"name":"fra_meter","type":["null","int"],"default":null},{"name":"til_meter","type":["null","int"],"default":null},{"name":"kortform","type":["null","string"],"default":null}]}]},{"name":"strekningslengde","type":["null","int"],"default":null}]}]}]},{"name":"relasjoner","type":["null",{"type":"record","name":"vegobjekt","namespace":"relasjoner","fields":[{"name":"ManglendeFelt","type":["null","string"],"default":null}]}]}]}]},{"type":["null",{"type":"long","logicalType":"timestamp-millis"}],"name":"AdmInsertUnixTime","default":null},{"name":"AdmPreviousUnixTime","default":null,"type":["null",{"type":"long","logicalType":"timestamp-millis"}]},{"name":"AdmPartitionKeyValue","type":["null","string"],"default":null},{"name":"AdmSrcFilename","default":null,"type":["null","string"]},{"type":["null","string"],"default":null,"name":"AdmId"},{"name":"AdmSource","default":null,"type":["null","string"]}]}"""

# verify if my_avro_schema_string is valid avro 
my_avro_schema = schema = parse(my_avro_schema_string)


In [5]:
# parse avro schema string to a org.apache.Schema object
java_avro_schema=spark._jvm.org.apache.avro.Schema.parse(my_avro_schema_string)

# convert to spark struct type
spark_java_struct_type=spark._jvm.com.databricks.spark.avro.SchemaConverters.toSqlType(java_avro_schema).dataType()

In [18]:

spark_json_string = spark_java_struct_type.json()

pyspark_schema = StructType.fromJson(json.loads(spark_json_schema))

In [20]:
pyspark_schema_create=StructType([StructField('Name', StringType(), True),
                     StructField('DateTime', TimestampType(), True),
                     StructField('Age', IntegerType(), True)])
pyspark_schema_create

StructType(List(StructField(Name,StringType,true),StructField(DateTime,TimestampType,true),StructField(Age,IntegerType,true)))

In [22]:
# The loaded parquet does not correspond to schema, thus "null", but shows the schema works
df = spark.read.schema(pyspark_schema).parquet("/tmp/fno_parquet_data/example")
df.show()


+----+---------+-----------------+-------------------+--------------------+--------------+-----+---------+
|type|vegobjekt|AdmInsertUnixTime|AdmPreviousUnixTime|AdmPartitionKeyValue|AdmSrcFilename|AdmId|AdmSource|
+----+---------+-----------------+-------------------+--------------------+--------------+-----+---------+
|null|     null|             null|               null|                null|          null| null|     null|
|null|     null|             null|               null|                null|          null| null|     null|
|null|     null|             null|               null|                null|          null| null|     null|
|null|     null|             null|               null|                null|          null| null|     null|
|null|     null|             null|               null|                null|          null| null|     null|
|null|     null|             null|               null|                null|          null| null|     null|
|null|     null|             null|   

In [12]:
spark.stop()